# Topic Retrieval Analytic

In this notebook, we will go through the steps needed to build the `.air` file which configures using the topic retrieval remote analytic.

## Package Installation and Import

First, we import the `aisquared` package.

In [1]:
# import
import aisquared

/Users/jwrenn4/miniforge3/lib/python3.9/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


## Configure the steps required

The workflow of using the `aisquared` package involves the creation of steps, in the following order:

- harvesting
- preprocessing
- analytic/model
- postprocessing
- rendering
- model_feedback
- feedback_steps

In [2]:
# Because this specific analytic doesn't require any inputs, the harvesting steps can be empty
harvester = []

# There are also no preprocessing steps for this analytic
preprocesser = None

# For the analytic step, we need the DeployedAnalytic class
analytic = aisquared.config.analytic.DeployedAnalytic(
    url = "https://e8qgxh1zp0.execute-api.us-east-2.amazonaws.com/test/returntrendingtopicsresource",
    input_type = 'text',
    secret = 'request',
    header = None
)

# No posptprocessing is required
postprocesser = None

# The DocumentRendering class is used for rendering
renderer = aisquared.config.rendering.DocumentRendering(
    prediction_key = 'topic',    # the key to primarily render
    words = 'keywords',          # the key to use to retrieve document keywords
    documents = 'docs',          # the key to use to retrieve key documents per topic
    include_probability = False  # whether to include probability scores
)

# SimpleFeedback used to get prediction feedback
model_feedback = aisquared.config.feedback.ModelFeedback()
model_feedback.add_question('Are any of the topics incorrect?', 'multiChoice', ['High growth', 'International stocks', 'Treasury bonds', 'Midcap stocks'])
model_feedback.add_question('Do you have any additional feedback about this model?', 'text')
feedback = aisquared.config.feedback.SimpleFeedback()

## Compile all Steps as a ModelConfiguration Object

After the steps have been initiated, the entire `ModelConfiguration` object can be constructed and the `.compile()` method run to create the `.air` file.

In [3]:
# Initialize the ModelConfiguration object
config = aisquared.config.ModelConfiguration(
    name = 'TopicRetriever',                      # the name of the model
    harvesting_steps = [harvester],               # the harvester
    preprocessing_steps = preprocesser,           # the preprocesser
    analytic = analytic,                          # the analytic
    postprocessing_steps = postprocesser,         # the postprocessing steps
    rendering_steps = renderer,                   # the rendering steps
    feedback_steps = [model_feedback, feedback]   # the prediction feedback
)

# Run the .compile() methods to create the .air file
config.compile()